In [ ]:
class LSTM_LanguageModel(object):
    def __init__(self,word_dim,hidden_dim,vocab_size,vocab):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.vocab = vocab
        self.initWord = np.eye(vocab_size)[0]
        
    def define_network(self):
        
        def lstm_step(prev_output,prev_hidden,prev_memory):
            input_gate = T.nnet.hard_sigmoid(T.dot(( self.U_input),prev_output) + T.dot(self.W_input,prev_state) + self.bias_input)
            forget_gate = T.nnet.hard_sigmoid(T.dot(( self.U_forget),prev_output) + T.dot(self.W_forget,prev_state)+ self.bias_forget)
            output_gate = T.nnet.hard_sigmoid(T.dot((self.U_output),prev_output) + T.dot(self.W_output,prev_state)+ self.bias_output)



            stabilized_input = T.tanh(T.dot((self.U),prev_output) + T.dot(self.W,prev_state) + self.bias)
            c = forget_gate * prev_content + input_gate * stabilized_input
            s = output_gate * T.tanh(c) 
            
            output = T.softmax(T.dot(self.W_o,s))[0]
            
            return [output,s,c,  theano.scan_module.until(np.argmax(output) > (self.vocab_size - 2)]
        
        
        [self.output,self.hidden_state, self.memory_content], updates = theano.scan(
            forward_step,
            truncate_gradient=-1,
            n_steps= 5,
            outputs_info=[None,dict(initial = self.InitWord),dict(initial= T.zeros(self.hidden_dim, dtype=theano.config.floatX)),
                          dict(initial=T.zeros(self.hidden_dim, dtype=theano.config.floatX)),
                          ])
            
        